In [1]:
import tensorflow as tf

from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

import numpy as np
from numpy import asarray
import pandas as pd
import random
import matplotlib.pyplot as plt

# Dataset exploration


*   Explore the **In-shop Clothes Retrieval Benchmark** of the **DeepFashion** dataset
http://mmlab.ie.cuhk.edu.hk/projects/DeepFashion/InShopRetrieval.html
*   Import the dataset in `Google Colab` instead of downloading the whole dataset locally
*   Visualize some samples from the dataset
*   Look for variations and correlations in the dataset
*   Use the `matplotlib` and `seaborn` library to visualize data



In [ ]:
!unzip "/content/drive/MyDrive/img.zip" -d "/content/drive/MyDrive/imgdataset"

In [ ]:
img_data = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/imgdataset/img',
    image_size=(256, 256)
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
plt.figure(figsize=(15, 15))
for images,labels in img_data.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.axis("off")
  
plt.show()

In [ ]:
pip install -U seaborn-image

In [ ]:
import seaborn_image as isns
isns.load_image()

g = isns.ImageGrid(cells, slices=[10, 20, 30, 40])

# ResNET Architecture

# New Section



*   Read about **ResNET** architecture and advantage of residual blocks
*   Build a simple 6-layer ConvNET and train it on **CIFAR-10** dataset
*   Introduce residual blocks in the above model and compare the results 
*   Use the `tensorflow` library to import ResNET model with imgaenet weights
*   Change some parameters and document the results
*   Visualize the output of some layers and add/delete layers to see the effect of individual layers on the model accuracy



# ResNET50 Model

In [29]:
import tensorflow as tf
from keras import layers
import keras.applications.resnet50
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input  


In [30]:
from tensorflow.keras import datasets
tf.keras.datasets.cifar10.load_data()
(x_train,y_train),(x_test,y_test) = datasets.cifar10.load_data()

In [ ]:
print(x_train.shape)

In [32]:
resnet50 = ResNet50(
  include_top=True,
  weights='imagenet',input_tensor=None,
  input_shape=None,
  classifier_activation='softmax',
)

resnet50.trainable = False

resnet_model = keras.Sequential([
    layers.UpSampling2D(size=(7, 7)),
    resnet50,
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax'),
])

In [33]:
resnet_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
resnet_model.fit(x_train, y_train, epochs=10)

In [ ]:
test_loss, test_acc = resnet_model.evaluate(x_train, y_train, verbose=1) 

print('Test accuracy:', test_acc)

In [ ]:
resnet_model.summary()

# Residual Blocks added

In [21]:
from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10

In [22]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [23]:
def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)
  
    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

def create_res_net():
    
    inputs = Input(shape=(32, 32, 3))
    
    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=64,
               padding="same")(t)
    t = relu_bn(t)
    
    num_blocks_list = [1]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j==0 and i!=0), filters=64)
    
    t = AveragePooling2D(4)(t)
    t = Flatten()(t)
    outputs = Dense(10, activation='softmax')(t)
    
    model = Model(inputs, outputs)

    model.compile(
        optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy']
    )

    return model


In [24]:
model = create_res_net()

In [ ]:
model.fit(x_train, y_train, epochs=15)

In [ ]:
test_loss, test_acc = model.evaluate(x_train, y_train, verbose=1) 

print('Test accuracy:', test_acc)

In [ ]:
model.summary()